In [33]:
# if lights are off when a color transition starts, or for a brightness transition to zero, transition will fail.
# So we need a way to initiate a transition based on expected state.

# But reworking the script to do that is... kinda big. This is the quick and dirty version.
# Assumes the transition is 1hr+ long. 

In [34]:
import datetime
import random
import time

from tradfri import Tradfri

Todo:
look for all 'todo' instances here.


... this is not turning out to be as 'quick and dirty' as I thought.


figure out multithreading. https://stackoverflow.com/questions/2846653/how-to-use-threading-in-python

#### Approach:

Process:
Plan transition for [brightness, color] for each light based on start & end times
Skip through transition until we reach current time. Do short transition (30s?) to that. 


In [52]:
### DEBUG
from pprint import pprint
DEBUG = 1

In [99]:
### CONFIG:

evening_start_time = datetime.time(hour=17, minute=30)

# 'quick' initial transition to bring lights in line with overall plan. think a minute-ish.
# definitely don't go more than 15 min or you're not 'catching up' fast enough.
initial_transition_duration = datetime.timedelta(seconds=6)

# different times for end of color transition and end of brightness transition:
evening_color_end_value = 2200
evening_color_end_time = datetime.time(hour=23)

evening_brightness_end_value = 0
evening_brightness_end_time = datetime.time(hour=23, minute=45)

# NOTE: don't transition over midnight, or this will break. Like I said, quick and dirty.
#(Modify to use datetimes if you need to transition spanning midnight.
#  Or check if start is after end & +1 the date.)


# we'll extend transitions by a random amount, between 0 and this many seconds,
#  to prevent everything from switching off at once.
# recommend 5 minutes = 300 seconds
random_delay_range = 300



In [37]:
# basically just a record of daytime values for the office. also defined in homeassistant as an automation.
# *not* the same names as in light-schedule.py; these are more like the entity ids.
daytime_values = {"office_table": {
                            "brightness": 230,
                            "color": 4000
                            },
           "desk_lamp": {
                           "brightness": 200,
                           "color": 3200
                       },
           "geo_desk": {
                           "brightness": 130,
                           "color": 3350
                       },
           "floor_uplight": {
                           "brightness": 230,
                           "color": 3800
                       },
           "monitor_left": {
                           "brightness": 130,
                           "color": 3700
                       },
           "monitor_right": {
                           "brightness": 130,
                           "color": 3700
                       }
          }


In [38]:
# Do some setup / initial math and stuff.

today = datetime.datetime.now().date()

# make datetimes from dates. Use these internally instead of vals above.
e_start = datetime.datetime.combine(today, evening_start_time)
c_end = datetime.datetime.combine(today, evening_color_end_time)
b_end = datetime.datetime.combine(today, evening_brightness_end_time)

# timedeltas
color_duration = c_end - e_start
brightness_duration = b_end - e_start

# put these into a more easily programmatically-navigable structure.
details = {
              "color": {
                        "value": evening_color_end_value,
                        "end_time": evening_color_end_time,
                        "duration": color_duration
                       },
              "brightness": {
                        "value":evening_brightness_end_value,
                        "end_time": evening_brightness_end_time,
                        "duration": brightness_duration
              }
}


In [39]:
# list of each transition we'll have to do. these are *start* values.
transitions = []
for i in daytime_values:
    #print(i)
    transitions.append({"name": i, "type": "brightness", "value": daytime_values[i]['brightness']})
    transitions.append({"name": i, "type": "color", "value": daytime_values[i]['color']})

In [40]:
transitions[0]

{'name': 'office_table', 'type': 'brightness', 'value': 230}

In [41]:
### multithreading demo here

In [42]:
def get_val(transition):
    entity = 'light.' + transition['name']
    # instantiate
    device = Tradfri(entity)
    
    if transition['type'] == 'brightness':
        val = device.get_brightness()
    else: # it's color
        val = device.get_temp_kelvin()

    #print("gv()", transition['name'], transition['type'], val, sep='\t')
    return (transition['name'], transition['type'], val)

In [43]:
# non-multithreaded version
for i in transitions[0:3]:
    val = get_val(i)
    #print(i['name'], i['type'], val[1])

In [44]:
from multiprocessing.dummy import Pool as ThreadPool
# maybe problematic if we have hundreds of transitions, but we don't
pool = ThreadPool(len(transitions))

In [45]:
# multithreaded version. this is actually pretty easy.
results = pool.map(get_val, transitions)

In [46]:
results

[('office_table', 'brightness', 230),
 ('office_table', 'color', 4000),
 ('desk_lamp', 'brightness', 200),
 ('desk_lamp', 'color', 3205),
 ('geo_desk', 'brightness', 130),
 ('geo_desk', 'color', 3356),
 ('floor_uplight', 'brightness', 230),
 ('floor_uplight', 'color', 3802),
 ('monitor_left', 'brightness', 130),
 ('monitor_left', 'color', 3704),
 ('monitor_right', 'brightness', 130),
 ('monitor_right', 'color', 3704)]

In [47]:
## end multithreading demo

In [96]:
# ok, here we go.

def catch_up(transition):
    # 'resumes' an existing transition.
    
    ### todo: function too big, split up?
    ### todo: check if current time is between start and end times before going through the whole plan.
    
    ### todo: make initial transition shorter if # of steps is low. min 1 second per step. 
    
    # BUG: plan_transition function transitions from *current* attrs. I think. I'm pretty sure.
    
    # BUG: 
    # error when transitioning a 'color' value.
    """
    ~\Documents\homeassistant\tradfri-lights\tradfri.py in calculate_steps_by_granularity(current_attrs, new_attr)
    281 
    282                 # skip if current state and transition state are equal:
--> 283                 if new_attr[key] == current_attrs[key]:

    KeyError: 'color'
    """
    
    # BUG:
    """
    ~\Documents\homeassistant\tradfri-lights\tradfri.py in execute_transition(self, plan)
    351         if self.debug > 0:
    352             print("time until start", time_until_start)
--> 353         time.sleep(time_until_start.total_seconds() - 0.1)
    354 
    355         step_sleep = MIN_STEP_DURATION.total_seconds()

    ValueError: sleep length must be non-negative
    """
    
    
    # step 1 in randomizing
    if DEBUG:
        time.sleep(random.uniform(0,1))
    else:
        time.sleep(random.uniform(0,9))
    

    entity = 'light.' + transition['name']
    # instantiate
    device = Tradfri(entity)
    
    
    #print(details[i['type']])
    transition_type = transition['type']
    final_attr = {transition_type: details[transition_type]['value']}
    duration = details[transition_type]['duration']
    
    if transition_type == "brightness":
        cur_value = {"brightness": device.get_brightness()}
    else:
        cur_value = {"color": device.get_color()}
    if DEBUG:
        print("cur value:", cur_value)
            
    
    # usage:
    # plan_transition(self, new_attr, duration, start_time=None)
    
    # randomize a bit more.
    random_delay = random.uniform(0, random_delay_range)
    duration += datetime.timedelta(seconds = random_delay)
    plan = device.plan_transition(final_attr, duration, e_start)
    if DEBUG:
        print("\trandomized delay:", random_delay)
        print("\tnew end time:", e_start + duration)
    
    # debug
    #pprint(plan['plan'][-3:])
    
    # find the point in the the transition *after* our initial transition to current value
    benchmark_time = datetime.datetime.now() + initial_transition_duration
    
    current_step_no = None
    # find out how far into the transition we are.
    for i in plan['plan']: 
        if i['step_start_time'] >= benchmark_time:
            current_step_no = i['step_number'] - 1
            break
    
    if current_step_no is None:
        # probably beyond the end of the transition.
        print("no step found!")
        
        ### write me
        
    elif current_step_no == 0:
        # may or may not have started yet? do we need to special case this?
        print("cur step is zero!")
        
        ### write me
        
    elif current_step_no < 0:
        # definitely haven't started yet
        print("cur step is negative! step:", current_step_no)
        
        ### write me
    
    else: #continue
        
        if DEBUG:
            total_steps = len(plan['plan'])
            pct_complete = round(100*(current_step_no/total_steps))
            print("current transition is", pct_complete, "percent completed")
    
        current_step_vals = plan['plan'][current_step_no]

        print("cur step:", current_step_vals)

        step_value = {transition_type: plan['plan'][current_step_no][transition_type]}


        if cur_value != step_value:
            # Do a quick transition to these values
            if DEBUG: print("catching up to the following values over the next", 
                initial_transition_duration.seconds, 
                "seconds:", step_value)
            device.transition(step_value, initial_transition_duration)
            if DEBUG: print("caught up.", end=" ")

        if DEBUG: print("resuming normal schedule")
        
        if DEBUG:
            print("csn #:", current_step_no, "of", total_steps)
        
        # Then, resume normal transition, starting from current step / values.
        new_plan = plan['plan'][current_step_no:]
        plan['plan'] = new_plan
        device.execute_transition(plan)
    

In [104]:
for i in transitions[0:14:2]:
    print("this cell i:", i)
    catch_up(i)

this cell i: {'name': 'office_table', 'type': 'brightness', 'value': 230}
cur value: {'brightness': 7}
	randomized delay: 122.0734258805052
	new end time: 2018-09-10 23:47:02.073426
no step found!
this cell i: {'name': 'desk_lamp', 'type': 'brightness', 'value': 200}
cur value: {'brightness': 2}
	randomized delay: 180.26558591083997
	new end time: 2018-09-10 23:48:00.265586
no step found!
this cell i: {'name': 'geo_desk', 'type': 'brightness', 'value': 130}
cur value: {'brightness': 1}
	randomized delay: 58.939678406048834
	new end time: 2018-09-10 23:45:58.939678
no step found!
this cell i: {'name': 'floor_uplight', 'type': 'brightness', 'value': 230}
cur value: {'brightness': 1}
	randomized delay: 210.2204994458713
	new end time: 2018-09-10 23:48:30.220499
no step found!
this cell i: {'name': 'monitor_left', 'type': 'brightness', 'value': 130}
cur value: {'brightness': 1}
	randomized delay: 62.12418304159849
	new end time: 2018-09-10 23:46:02.124183
no step found!
this cell i: {'name

ValueError: sleep length must be non-negative

In [ ]:
def planner(transition):
    # half ass extracted this from the catch_up(). it's still there too.
    
    entity = 'light.' + transition['name']
    # instantiate
    device = Tradfri(entity)
    
    #print(details[i['type']])
    transition_type = transition['type']
    final_attr = {transition_type: details[transition_type]['value']}
    duration = details[transition_type]['duration']
    
    
    # usage:
    # plan_transition(self, new_attr, duration, start_time=None)
    
    # randomize a bit more.
    duration + datetime.timedelta(seconds = random.uniform(0, random_delay_range))
    plan = device.plan_transition(final_attr, duration, e_start)
    
    return plan

In [85]:
for i in transitions[0:1]:
    plan = planner(i)
    print("steps:", len(plan['plan']))
pprint(plan['details'])
pprint(plan['plan'][0:5])
print("...")
pprint(plan['plan'][-5:])

steps: 7
{'entity_id': 'light.office_table',
 'start_time': datetime.datetime(2018, 9, 10, 17, 30),
 'start_value': 7,
 'step_change': -1.0,
 'step_duration': datetime.timedelta(0, 3214, 285714),
 'steps': 7,
 'target_value': 0,
 'transition_type': 'brightness'}
[{'brightness': 6.0,
  'step_number': 0,
  'step_start_time': datetime.datetime(2018, 9, 10, 17, 30)},
 {'brightness': 5.0,
  'step_number': 1,
  'step_start_time': datetime.datetime(2018, 9, 10, 18, 23, 34, 285714)},
 {'brightness': 4.0,
  'step_number': 2,
  'step_start_time': datetime.datetime(2018, 9, 10, 19, 17, 8, 571428)},
 {'brightness': 3.0,
  'step_number': 3,
  'step_start_time': datetime.datetime(2018, 9, 10, 20, 10, 42, 857142)},
 {'brightness': 2.0,
  'step_number': 4,
  'step_start_time': datetime.datetime(2018, 9, 10, 21, 4, 17, 142856)}]
...
[{'brightness': 4.0,
  'step_number': 2,
  'step_start_time': datetime.datetime(2018, 9, 10, 19, 17, 8, 571428)},
 {'brightness': 3.0,
  'step_number': 3,
  'step_start_tim

In [86]:
dorp = [i for i in range(8)]

In [87]:
dorp

[0, 1, 2, 3, 4, 5, 6, 7]

In [88]:
dorp[0:8:2]

[0, 2, 4, 6]